In [ ]:
#import kagglehub

# Download latest version
#path = kagglehub.dataset_download("corrieaar/apartment-rental-offers-in-germany")

#print("Path to dataset files:", path)

# in order to download effectively, it was necessary to create a Token for the Kaggle API and place it in the folder alvarezsaav/.kaggle,
#moving it from the downloads folder.

# The data were downloaded to this folder after the process was complete:

#!ls /home/alvarezsaav/.cache/kagglehub/datasets/corrieaar/apartment-rental-offers-in-germany/versions/6/immo_data.csv

# We then moved it to our working folder here.

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 93.0M/93.0M [01:07<00:00, 1.45MB/s]

Extracting files...


Path to dataset files: /home/alvarezsaav/.cache/kagglehub/datasets/corrieaar/apartment-rental-offers-in-germany/versions/6


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import os
import sys
import sklearn

import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [ ]:
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path_files='/content/gdrive/MyDrive/Colab Notebooks/'

In [ ]:
df=pd.read_csv(path_files + 'immo_data.csv')

## Converting text variables to structural data

### For this purpose we use open AI api module.


In [1]:
#!openai migrate


In [ ]:

import openai

In [ ]:
blank_descriptions = df['description'].isna()| df['description'].str.strip().eq('')
blank_facilities = df['facilities'].isna()| df['facilities'].str.strip().eq('')

In [ ]:
df_blankdesc=df[blank_descriptions]
df_blankfaci=df[blank_facilities]

df2=df[~blank_facilities]
df3=df2[~blank_descriptions]

df_nonblank=df3

<ipython-input-6-b21411d07a54>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df3=df2[~blank_descriptions]


In [ ]:
len(df3['facilities'].isna()| df3['facilities'].str.strip().eq(''))

202761

In [ ]:
df_text=df[['description', 'facilities']]

We will need this particular version of OpenAI module to work and interact with the API.

In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
import openai

# Set the API key
openai.api_key = "INSERT_YOUR_PRIVATE_KEY"

# This is our prompt / query:

def analyze_text(description, facilities):
    prompt = f"""
    Derive sentiments from rental properties descriptions {description} and facilities {facilities}.
    Return a Python only nummerical list [luxury, tranquility, safety, accessibility, security, overall sentiment] each rated 0-9.
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        #prompt=prompt,
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=200

    )

    # Extracting the text result from the response
    extracted_features = response['choices'][0]['message']['content'].strip()

    return extracted_features



In [ ]:
import re

In [ ]:
numbers=[]
for s in features:
  nums=re.findall(r'\d+', s)
  numbers.extend([int(num) for num in nums])

In [ ]:
# Example usage
description = "Wohnung in ruhiger Lage mit viel Tageslicht."
facilities = "Parkplatz, Balkon, Aufzug."

features = analyze_text(description, facilities)
print(features)

[5, 8, 7, 9, 6, 7]


In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.5 MB/s eta 0:00:00


In [ ]:
list_indexes_with_comments=np.unique(df3.index)

In [ ]:
list_indexes_with_comments=np.unique(df3.index)



#list_indexes_with_comments=list_indexes_with_comments[0:100000]

### VARIABLE CREATION:

for i in list_indexes_with_comments:
  original_index = i
  try:

    ## SENTIMENT ANALYSIS PER OBJECT:
    description = df3['description'][i]
    facilities = df3['facilities'][i]

    features=analyze_text(description, facilities)

    ## WE sort the list that ChatGPT produces (optional):
    #numeric_list=[]
    #numeric_list=features
    #for s in features:
    #  nums=re.findall(r'\d+', s)
    #  numbers.extend([int(num) for num in nums])
    #numeric_list=[int(num) for string in features for num in re.findall(r'\d+', features)]

    new_list=[features]
    new_list.append(i)

    ## We write into an external file:
    with open(path_files + 'sentiment_analysis.txt', mode='a', newline='') as csvfile:
        #writer = csv.writer(csvfile, delimiter=",")
        #writer.writerow(features)
        #csvfile.write(','.join(str(i) for i in numbers))
        writer = csv.writer(csvfile)
        writer.writerow(new_list)
  except:
    print ('some issue with index ' + str(i))
    except_list=[[0,0,0,0,0]]
    except_list.append(i)
    with open(path_files + 'sentiment_analysis.txt', mode='a', newline='') as csvfile:
        #writer = csv.writer(csvfile, delimiter=",")
        #writer.writerow(features)
        #csvfile.write(','.join(str(i) for i in numbers))
        writer = csv.writer(csvfile)
        writer.writerow(except_list)
        pass

### We passed a try - except in case there is some error that stops the code in some line, so that it keeps moving forward.